In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/YES_BANK.xlsx


In [2]:
import os
import numpy as np
import pandas as pd
import random 
import copy
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import mean_absolute_error as MAE
from sklearn import linear_model as lm
from keras.models import Sequential
from keras.layers import Activation, Dense,Bidirectional,Dropout,LSTM

Using TensorFlow backend.


In [3]:
 original_data= pd.read_excel('/kaggle/input/YES_BANK.xlsx',
header=None,
index_col=False,
keep_default_na=True,names=['Date','Open','High','Low','Close','Volume']
)


In [4]:
original_data.shape

(3632, 6)

In [5]:

original_data['Date'] = pd.to_datetime(original_data['Date'])
original_data.head()

,Date,Open,High,Low,Close,Volume
0,2020-03-09,17.0,22.8,16.20,21.25,69542810
1,2020-03-06,33.2,33.2,5.55,16.20,106336244
2,2020-03-05,29.4,37.9,28.05,36.85,43386506
3,2020-03-04,31.4,31.7,28.65,29.30,10055149
4,2020-03-03,32.8,33.3,30.60,31.25,10299879


In [8]:
def datasetpreparation(original_data):
  data_split = int(len(original_data)*0.8)
  data1 = original_data.drop(['Date','Volume'],axis = 1)
  data1.head()
  data1_input = []
  for i in range(len(data1) - 4):
    list1 = data1[i:i+5].values.tolist()
    data1_input.append(list1)
  #print(np.array(data4_input).shape)
  #(data4_input)
  data1_output = data1.ix[5:,:]
  #data4_output
  train1_input = np.array(data1_input[0:data_split])
  train1_output = np.array(data1_output[0:data_split])
  print(train1_input.shape)
  print(train1_output.shape)
  test1_input = np.array(data1_input[data_split:])
  test1_output = np.array(data1_output[data_split:])
  print(test1_input.shape)
  print(test1_output.shape)
  return train1_input,train1_output,test1_input,test1_output

In [9]:
def LSTMmodel():
  model1 = Sequential()
  model1.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(5, 4)))
  model1.add(Dropout(0.25))
  model1.add(Dense(4))
  model1.compile(optimizer='adam', loss='mse')
  return model1

In [10]:
def modeltrain(train1_input,train1_output,model1):
  #train_input,train_output,test1_input,test1_output = datasetpreparation(original_data)
  #model = LSTMmodel 
  history = model1.fit(train1_input, train1_output, epochs=500, verbose=1)
  model1.save('project_model.h5')

In [11]:
def evaluateval(model1,train1_input,train1_output):
  #train_input,train_output,test1_input,test1_output = datasetpreparation(original_data)
  predict = model1.predict(train1_input)
  predict1 = pd.DataFrame(predict)
  output = pd.DataFrame(train1_output)
  for i in range(4):
    plt.plot(predict1.ix[:,i])
    plt.plot(output.ix[:,i])
    plt.show()

In [ ]:
def test(model1,train1_input,train1_output,test1_input,test1_output,i):
  input = test1_input[i].reshape(1,5,4)
  train1_input = np.concatenate((train1_input,input),axis = 0)
  output = test1_output[i].reshape(1,4)
  train1_output = np.concatenate((train1_output,output),axis = 0)
  prediction = model1.predict(input)
  return prediction,train1_input,train1_output

In [ ]:
def feedback(model1,train1_input,train1_output,test1_input,i):
  model1.fit(train1_input, train1_output, epochs=10, verbose=1)
  prediction = model1.predict(test1_input[i].reshape(1,5,4))
  return prediction

In [ ]:
def runprograme(original_data):
  train1_input,train1_output,test1_input,test1_output = datasetpreparation(original_data)
  model1 = LSTMmodel()
  modeltrain(train1_input,train1_output,model1)
  from keras.models import load_model
  model1 = load_model('project_model.h5')
  evaluateval(model1,train1_input,train1_output)
  for i in range(2):
   pred,train1_input,train1_output = test(model1,train1_input,train1_output,test1_input,test1_output,i)
   prediction = feedback(model1,train1_input,train1_output,test1_input,i)
   print(pred)
   print(prediction)
   print(test1_output[i])

In [ ]:
runprograme(original_data)